In [80]:
#hide
! [ -e /content ] && pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz
import fastbook
fastbook.setup_book()

#hide
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 16

In [81]:
cwd = Path.cwd()
cwd

Path('/home/jupyter/fastbook')

In [116]:
data = Path(cwd,'mydata','county.gzip')
df = pd.read_parquet(data)
df_cty_names = df[['FIPS','Area']].drop_duplicates().set_index('FIPS')
df = df.drop('Area', axis=1)
df['Year'] = df['Year'].astype(str) + '-12-31'
df['Year'] = pd.to_datetime(df['Year']).dt.to_period('Y')

In [117]:
df = df.iloc[:,:5]

In [125]:
data_st = Path(cwd,'mydata','states.gzip')
ds = pd.read_parquet(data_st)
#df = df.drop('Area', axis=1)
ds['Year'] = ds['Year'].astype(str) + '-12-31'
ds['Year'] = pd.to_datetime(ds['Year']).dt.to_period('Y')
ds = ds.iloc[:,:4]

In [240]:
max(ds['Year'].unique())

Period('2021', 'A-DEC')

In [241]:
states = list(ds['Area'].unique())

In [242]:
ds_AL = ds[ds['Area']=='Alabama']

In [248]:
values = 'GDP'
ds_AL_pv = ds_AL.pivot(index='Year', columns='Sector', values=values)  #.pct_change()
if values == 'GDP':
    ds_AL_pv = ds_AL_pv[:-1]

In [249]:
ds_AL_pct_1 = ds_AL_pv.pct_change(periods=1).dropna()
ds_AL_pct_1

Sector,Construction,Education and health services,Financial activities,Government,Information,Leisure and hospitality,Manufacturing,Natural resources and mining,Other services,Professional and business services,"Total, Govt and Private","Trade, transportation, and utilities"
Year,,,,,,,,,,,,
2003,0.039849,0.035275,0.042859,0.062083,-0.034015,0.038262,0.023625,0.382424,0.022181,0.106643,0.052136,0.041492
2004,0.058175,0.076116,0.078920,0.053258,0.098638,0.036035,0.212875,0.243919,0.092938,0.096246,0.101099,0.066863
2005,0.130823,0.050910,0.098382,0.057706,0.080633,0.091516,0.057904,0.078773,0.085238,0.069646,0.075359,0.113256
2006,0.096383,0.070835,0.010805,0.045222,-0.464857,0.068196,0.054300,-0.040281,-0.187172,0.061337,0.047985,0.069718
2007,0.081337,0.038216,0.062506,0.080431,0.763331,0.042574,-0.012451,0.096239,0.286693,0.078636,0.039083,0.005286
2008,-0.087082,0.067566,-0.058509,0.036643,0.024634,0.007114,0.002207,0.067445,-0.004983,0.094657,0.008967,0.033290
2009,-0.114875,0.052329,0.048614,0.050297,-0.076100,0.013146,-0.132791,-0.196246,0.004525,-0.001772,-0.020601,-0.034264
2010,-0.079750,0.047805,0.041629,0.015974,-0.014116,0.043304,0.089153,0.072833,0.029857,0.033540,0.036964,0.043459
2011,-0.026664,0.037624,0.025129,0.018436,-0.019121,0.048039,0.142084,-0.048497,0.012590,0.041701,0.037616,0.028332


In [255]:
ds_AL__future_3yr_GDP_growth = ds_AL_pv.pct_change(periods=3)['Total, Govt and Private'].shift(periods=-3)
ds_AL__future_3yr_GDP_growth

Year
2002    0.245809
2003    0.240893
2004    0.171005
2005    0.098708
2006    0.026803
2007    0.024709
2008    0.053805
2009    0.107146
2010    0.098944
2011    0.073350
2012    0.069364
2013    0.064591
2014    0.089557
2015    0.107102
2016    0.114788
2017    0.054911
2018         NaN
2019         NaN
2020         NaN
Freq: A-DEC, Name: Total, Govt and Private, dtype: float64

In [190]:
ds_AL_pct = ds_AL_pv.pct_change().dropna()
ds_AL_pct = ds_AL_pct.melt(value_name='GDP pct_chg', ignore_index=False).reset_index()

In [199]:
ds_AL_2 = ds_AL.merge(ds_AL_pct, on=['Sector','Year'])

In [213]:
ds_AL_2

,Area,Sector,Year,GDP,GDP pct_chg
0,Alabama,Construction,2003,6448293000,0.039849
1,Alabama,Construction,2004,6823423000,0.058175
2,Alabama,Construction,2005,7716083000,0.130823
3,Alabama,Construction,2006,8459784000,0.096383
4,Alabama,Construction,2007,9147878000,0.081337
...,...,...,...,...,...
223,Alabama,"Trade, transportation, and utilities",2017,37295444000,0.021076
224,Alabama,"Trade, transportation, and utilities",2018,38870084000,0.042221
225,Alabama,"Trade, transportation, and utilities",2019,39546548000,0.017403
226,Alabama,"Trade, transportation, and utilities",2020,40571498000,0.025918


In [263]:
dx = ds_AL_2.pivot(index=['Year'], columns='Sector', values=['GDP pct_chg'])
dx.columns = [' -'.join(col).strip() for col in dx.columns.values]

In [264]:
dx['3yr_F_GDP'] = ds_AL__future_3yr_GDP_growth

In [265]:
dx

,GDP pct_chg -Construction,GDP pct_chg -Education and health services,GDP pct_chg -Financial activities,GDP pct_chg -Government,GDP pct_chg -Information,GDP pct_chg -Leisure and hospitality,GDP pct_chg -Manufacturing,GDP pct_chg -Natural resources and mining,GDP pct_chg -Other services,GDP pct_chg -Professional and business services,"GDP pct_chg -Total, Govt and Private","GDP pct_chg -Trade, transportation, and utilities",3yr_F_GDP
Year,,,,,,,,,,,,,
2003,0.039849,0.035275,0.042859,0.062083,-0.034015,0.038262,0.023625,0.382424,0.022181,0.106643,0.052136,0.041492,0.240893
2004,0.058175,0.076116,0.078920,0.053258,0.098638,0.036035,0.212875,0.243919,0.092938,0.096246,0.101099,0.066863,0.171005
2005,0.130823,0.050910,0.098382,0.057706,0.080633,0.091516,0.057904,0.078773,0.085238,0.069646,0.075359,0.113256,0.098708
2006,0.096383,0.070835,0.010805,0.045222,-0.464857,0.068196,0.054300,-0.040281,-0.187172,0.061337,0.047985,0.069718,0.026803
2007,0.081337,0.038216,0.062506,0.080431,0.763331,0.042574,-0.012451,0.096239,0.286693,0.078636,0.039083,0.005286,0.024709
2008,-0.087082,0.067566,-0.058509,0.036643,0.024634,0.007114,0.002207,0.067445,-0.004983,0.094657,0.008967,0.033290,0.053805
2009,-0.114875,0.052329,0.048614,0.050297,-0.076100,0.013146,-0.132791,-0.196246,0.004525,-0.001772,-0.020601,-0.034264,0.107146
2010,-0.079750,0.047805,0.041629,0.015974,-0.014116,0.043304,0.089153,0.072833,0.029857,0.033540,0.036964,0.043459,0.098944
2011,-0.026664,0.037624,0.025129,0.018436,-0.019121,0.048039,0.142084,-0.048497,0.012590,0.041701,0.037616,0.028332,0.073350


In [145]:
ds_AL_pv.div(ds_AL_pv['Total, Govt and Private'], axis=0)

Sector,Construction,Education and health services,Financial activities,Government,Information,Leisure and hospitality,Manufacturing,Natural resources and mining,Other services,Professional and business services,"Total, Govt and Private","Trade, transportation, and utilities"
Year,,,,,,,,,,,,
2002,0.048635,0.069932,0.168748,0.166953,0.035301,0.025937,0.166134,0.020237,0.027600,0.077538,1.0,0.176265
2003,0.048067,0.068812,0.167260,0.168532,0.032410,0.025595,0.161632,0.026590,0.026814,0.081555,1.0,0.174482
2004,0.046193,0.067250,0.163891,0.161209,0.032338,0.024083,0.178040,0.030039,0.026615,0.081195,1.0,0.169057
2005,0.048576,0.065721,0.167400,0.158563,0.032496,0.024445,0.175150,0.030134,0.026860,0.080764,1.0,0.175015
2006,0.050819,0.067154,0.161461,0.158145,0.016594,0.024916,0.176206,0.027596,0.020833,0.081793,1.0,0.178644
2007,0.052885,0.067098,0.165101,0.164438,0.028160,0.025000,0.167467,0.029114,0.025797,0.084906,1.0,0.172834
2008,0.047851,0.070995,0.154059,0.168948,0.028597,0.024954,0.166345,0.030802,0.025440,0.092117,1.0,0.177000
2009,0.043245,0.076282,0.164947,0.181178,0.026977,0.025814,0.147290,0.025278,0.026093,0.093888,1.0,0.174531
2010,0.038378,0.077079,0.165689,0.177511,0.025648,0.025972,0.154703,0.026152,0.025914,0.093578,1.0,0.175624
